In [ ]:
import gurobipy as gp
from gurobipy import GRB

# ───── 1. 데이터 정의 ─────
K = [0, 1]   # OD pair set
N = [1, 2, 3, 4]   # Node set
A = [(1,2), (1,4), (2,3), (4,3), (2,4)]   # Arc set (tuples like (i,j))

q = {0: 10, 1: 15}   # demand of k
o = {0: 1, 1: 1}   # origin of k
t = {0: 3, 1: 3}   # destination of k
S = {0: 3, 1: 4}   # scheduled arrival time
z = {0: 25, 1: 30}   # shortest path distance
C = {(1,2): 20, (1,4): 10, (2,3): 20, (2,4): 10, (4,3): 15}   # capacity per arc
L = {(1,2): 10, (1,4): 20, (2,3): 15, (2,4): 10, (4,3): 5}   # length per arc
B = {(1,2): 1, (1,4): 1, (2,3): 1, (2,4): 1, (4,3): 1}   # operational status of arcs
tau = 30      # 30분 time step
v_avg = 60    # km/h → 1분에 1km → 30분이면 30km 가능
epsilon = 1
M = 1e5
w1, w2 = 1, 1

# ───── 2. 모델 생성 ─────
model = gp.Model("DelayAwareFlow")

# ───── 3. 변수 정의 ─────
f = model.addVars(K, A, name="f", lb=0)
u = model.addVars(K, name="u", lb=0)
h = model.addVars(K, vtype=GRB.BINARY, name="h")
s = model.addVars(K, vtype=GRB.INTEGER, lb=0, name="s")
theta = model.addVars(K, vtype=GRB.BINARY, name="theta")
delta = model.addVars(K, vtype=GRB.BINARY, name="delta")
a = model.addVars(K, name="a", lb=0)
d = model.addVars(K, name="d", lb=0)
g = model.addVars(K, name="g", lb=0)

# ───── 4. 목적함수 ─────
model.setObjective(
    gp.quicksum(u[k] for k in K) +
    w1 * gp.quicksum(delta[k] for k in K) +
    w2 * gp.quicksum(1 - h[k] for k in K),
    GRB.MINIMIZE
)

# ───── 5. 제약식 추가 ─────

# (1) Flow conservation
for k in K:
    for i in N:
        inflow = gp.quicksum(f[k, j, i] for j, ii in A if ii == i)
        outflow = gp.quicksum(f[k, i, j] for ii, j in A if ii == i)
        if i == o[k]:
            model.addConstr(outflow - inflow == q[k] - u[k])
        elif i == t[k]:
            model.addConstr(outflow - inflow == u[k] - q[k], name=f"theta_{k}")
        else:
            model.addConstr(outflow - inflow == 0)

# (2) Capacity
for i, j in A:
    model.addConstr(gp.quicksum(f[k, i, j] for k in K) <= B[i, j] * C[i, j])
for k in K:
    for i, j in A:
        model.addConstr(f[k, i, j] <= h[k] * s[k])

# (3) Travel distance limit
for k in K:
    model.addConstr(
        gp.quicksum(L[i, j] * f[k, i, j] for i, j in A)
        <= (z[k] + v_avg * epsilon) * gp.quicksum(f[k, i, j] for i, j in A if i == o[k])
    )

# (4) Delay time constraints
for k in K:
    model.addConstr(g[k] >= q[k] * h[k] - M * (1 - theta[k]))
    model.addConstr(g[k] + epsilon <= q[k] * h[k] + M * theta[k])
    model.addConstr(theta[k] <= h[k])
    model.addConstr(d[k] >= a[k] - S[k])
    model.addConstr(d[k] <= M * delta[k])
    model.addConstr(d[k] >= epsilon * delta[k])

# (5) Time step limit (optional upper bound)
model.addConstr(
    gp.quicksum(L[i, j] for k in K for i, j in A) <= v_avg * tau / 60
)

# ───── 6. Solve ─────
model.optimize()

# ───── 7. 결과 출력 ─────
if model.status == GRB.OPTIMAL:
    print(f"\n[✓] Optimal Objective Value: {model.ObjVal}\n")

    print("=== Flow (f[k, i, j]) ===")
    for k in K:
        for i, j in A:
            val = f[k, i, j].X
            if val > 1e-6:  # 출력 기준 설정 (0에 가까운 건 생략)
                print(f"f[{k}, {i}->{j}] = {val:.2f}")

    print("\n=== Service Variables ===")
    for k in K:
        print(f"OD {k}: u = {u[k].X:.2f}, h = {int(h[k].X)}, s = {int(s[k].X)}, theta = {int(theta[k].X)}")

    print("\n=== Delay Information ===")
    for k in K:
        print(f"OD {k}: a = {a[k].X:.2f}, d = {d[k].X:.2f}, delta = {int(delta[k].X)}, g = {g[k].X:.2f}")
else:
    print("[✗] No optimal solution found.")


Set parameter Username
Set parameter LicenseID to value 2637066
Academic license - for non-commercial use only - expires 2026-03-16
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 28 rows, 26 columns and 72 nonzeros
Model fingerprint: 0x7dbeb382
Model has 10 quadratic constraints
Variable types: 18 continuous, 8 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
[✗